### Import modules and initialize

In [1]:
import os
import ee
import folium
import shapefile
import numpy as np

In [2]:
ee.Initialize()

### Inputs

In [3]:
TILES_DIR = '../../data/ghana/tiles'
SHP_DIR_LIST = os.listdir(TILES_DIR)
shp_list = []
for SHP_DIR in SHP_DIR_LIST:
    shp = os.path.join(TILES_DIR, SHP_DIR, SHP_DIR+'.shp')
    shp_list.append(shp)
shp = '../../data/ghana/tiles/ghana_l_1/ghana_l_1.shp'

### Collect imagery

In [4]:
# function to convert shapefile into feature collection
def convert_shp2feature (shp_fp):
    shp = shapefile.Reader(shp_fp)
    fields = shp.fields[1:]
    field_names = [field[0] for field in fields]
    wgs84 = ee.Projection('EPSG:4326')
    features = []
    for sr in shp.shapeRecords():
        atr = dict(zip(field_names, sr.record))
        geom = sr.shape.__geo_interface__
        ee_geometry = ee.Geometry(geom,'EPSG:4326')
        feat = ee.Feature(ee_geometry, atr)
        features.append(feat)
    fc = ee.FeatureCollection(features)
    feature = ee.Feature(fc.first())
    feature.set('tile', os.path.basename(shp_fp)[:-4])
    return feature

# function to cloud mask sentinel imagery
def mask_clouds_sentinel (image):
    qa = image.select('QA60')
    cloudBitMask = int(2**10)
    cirrusBitMask = int(2**11)
    clear_pixels = qa.bitwiseAnd(cloudBitMask).eq(0)\
        .And(qa.bitwiseAnd(cirrusBitMask).eq(0))
        
    return image.updateMask(clear_pixels)

# function to calculate centroid of feature
def calc_centroid (feature):
    # obtain geometry
    bounds = feature.geometry()
    
    # calculate centroid
    xmax = bounds.coordinates().getInfo()[0][2][0]
    xmin = bounds.coordinates().getInfo()[0][0][0]
    ymin = bounds.coordinates().getInfo()[0][3][1]
    ymax = bounds.coordinates().getInfo()[0][1][1]
    xmean = np.average([xmin, xmax])
    ymean = np.average([ymin, ymax])
    
    return [ymean, xmean], ee.Geometry.Point([xmean, ymean]), [[xmin, ymin], [xmin, ymax], [xmax, ymax], [xmax, ymin]]

In [5]:
ee_bounds = convert_shp2feature(shp)

In [6]:
poi, centroid, coord_bounds = calc_centroid(ee_bounds)

In [7]:
sentinel_sr_collection = image = ee.ImageCollection('COPERNICUS/S2_SR')
median_image = sentinel_sr_collection\
    .filterBounds(ee.Geometry.Polygon(coord_bounds))\
    .map(mask_clouds_sentinel)\
    .median()\
    .clip(ee_bounds)

### Display

Skip if not testing

In [8]:
# function to display ee image tiles on folium map
def add_ee_layer(self, eeImageObject, visParams, name):
  mapID = ee.Image(eeImageObject).getMapId(visParams)
  folium.raster_layers.TileLayer(
    tiles = "https://earthengine.googleapis.com/map/"+mapID['mapid']+"/{z}/{x}/{y}?token="+mapID['token'],
    attr = "Map Data &copy; <a href='https://earthengine.google.com/'>Google Earth Engine</a>",
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

In [9]:
# create map
folium.Map.add_ee_layer = add_ee_layer
eeMap = folium.Map(location=poi, zoom_start=9)

In [10]:
# define visualization parameters
sentinel_sr_vis_params = {
    'bands':['B4','B3','B2'], 
    'min':0, 
    'max':3000
}

In [11]:
# add layers to map
eeMap.add_ee_layer(median_image, sentinel_sr_vis_params, 'median_image')

In [12]:
# display map
eeMap.add_child(folium.LayerControl())
folium.Marker(poi).add_to(eeMap)
display(eeMap)

### Save to Google Drive

In [13]:
coord_bounds = ee_bounds.geometry().coordinates().getInfo()[0][:4]

task_config = {
    'folder': 'ghana',
    'scale': 10,
    'region': coord_bounds,
    'maxPixels': 10000000000
}

median_image = median_image.clip(ee_bounds)
task = ee.batch.Export.image(median_image, os.path.basename(shp)[:-4], task_config)

In [14]:
task.start()